In [1]:
import numpy as np
import time
import math
from numpy.linalg import qr
from numpy.linalg import eig
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from scipy.linalg.interpolative import estimate_spectral_norm

In [2]:
np.random.seed(1)
rng = np.random.RandomState(1999)
d = 3
n = 1000
X = rng.randn(d, n) + 2
X = X.reshape(n,d)

## PCA using sklearn

In [3]:
def traditional_pca(X):
    pca = PCA()
    pca.fit(X)
    return pca

In [4]:
## used to compare DBPCA with traditional PCA
def sklearn_pca_per_window(B, X, k, d, profiling=False):
    """
    B : int
        block size or window size
    X : numpy array n x d
        n is the number of data points and d is the number of attributes
    output q = estimated eigen vectors of cov(X)
    k : int
        number of principal components
    f : float
        forgetting factor (0,1], 1 basically means no forgetting factor
    """
    start_time = time.time()
    pcas = list()
    Ys = list()
    n = X.shape[0]
    for i in range(int(n/B)):
        start = i*B
        end = (i*B)+B
        W = X[start:end,:]
        pca = PCA()
        pca.fit(W)
        Y = pca.transform(W)
        pcas.append(pca)
        Ys.append(Y)
        
    if profiling:
        execution_time = time.time() - start_time ## return time in nanosecond
        return pcas, Ys, execution_time
        
    return pcas, Y

In [5]:
X = X.reshape(n, d)
pca = traditional_pca(X)
X.shape

(1000, 3)

In [6]:
pca.n_features_in_

3

In [7]:
pca.n_components_

3

In [8]:
Y = pca.components_
Y.shape

(3, 3)

In [9]:
## Project x on the principal components
reduced_X = pca.transform(X)
reduced_X

array([[-1.09737919,  1.02337033, -0.11475864],
       [-0.55413522, -0.03185881,  0.18405721],
       [-1.34500491, -0.42872754,  0.03845774],
       ...,
       [-0.55085044,  1.57277561, -0.1119256 ],
       [ 1.95726631, -0.96695499, -1.18815944],
       [-0.01062   ,  0.48998912, -0.90431295]])

In [10]:
back_X = pca.inverse_transform(reduced_X)
back_X

array([[1.68251986, 2.69206233, 0.71562236],
       [2.39334583, 2.21180288, 1.58854318],
       [2.94547341, 2.90643341, 1.45884194],
       ...,
       [1.04565381, 2.25293917, 0.66957513],
       [1.3170083 , 1.53965394, 4.31878756],
       [1.21058426, 2.55385917, 2.07470798]])

In [11]:
np.cov(X.T)

array([[ 0.98768423,  0.02860169, -0.01309278],
       [ 0.02860169,  1.00540082, -0.03614379],
       [-0.01309278, -0.03614379,  1.0004561 ]])

## Manual PCA

In [12]:
def compute_principal_components(cov, C, variance_explained=1, profiling=False):
    eig_values, eig_vectors = eig(cov)
    k = len(eig_values)
    if variance_explained != 1:
        trace = sum(eig_values)
        var_explained = 0
        for i, val in enumerate(eig_values):
            var_explained += val/trace
#             print(f'{i} - {var_explained}')
            if var_explained >= variance_explained:
                k = i
    return eig_vectors, k

In [13]:
def project_data(C, eig_vectors, k, profiling=False):
    """
    C is a matrix n x d, where n is number of data points, d is number of attributes
    eig_vectors of d x k
    """
    # now project X 
    Y = eig_vectors[:,0:k].T.dot(C.T)
    return Y

In [14]:
def do_pca(C, variance_explained=1, profiling=False):
    """
    C is a matrix n x d, where n is number of data points, d is number of attributes
    """
    cov = np.cov(C.T)
    eig_vectors, k = compute_principal_components(cov, C, variance_explained, profiling)
    Y = project_data(C, eig_vectors, k, profiling)
    return Y, eig_vectors, k

In [15]:
def reconstruct_X(Y, eig_vectors, k):
    return eig_vectors[:,0:k].dot(Y)

In [16]:
def project_a_data_point(x, eig_vectors, k):
    """
    x is a vector of length d, d is number of attributes
    eig_vectors of d x k
    """
    # now project X 
    y = eig_vectors[:,0:k].T.dot(x)
    return y

In [17]:
def reconstruct_x(y, eig_vectors, k):
    """
    y is a vector of length k, k is the number of principal components
    eig_vectors of d x k
    """
    return eig_vectors[:,0:k].dot(y)

In [18]:
## used to compare DBPCA with traditional PCA
def naive_pca_per_window(B, X, k, d, profiling=False):
    """
    B : int
        block size or window size
    X : numpy array n x d
        n is the number of data points and d is the number of attributes
    output q = estimated eigen vectors of cov(X)
    k : int
        number of principal components
    f : float
        forgetting factor (0,1], 1 basically means no forgetting factor
    """
    start_time = time.time()
    eig_vectors = list()
    Ys = list()
    Ms = list()
    n = X.shape[0]
    for i in range(int(n/B)):
        start = i*B
        end = (i*B)+B
        W = X[start:end,:]
        M = np.mean(W, axis=0) # first pass on X
        C = W - M              # second pass on X
        Y, eig_vector, k = do_pca(C) ## Y.T should be the same as reduced_X above
        Ys.append(Y)
        eig_vectors.append(eig_vector)
        Ms.append(M)
        
    if profiling:
        execution_time = time.time() - start_time ## return time in nanosecond
        return eig_vectors, Ys, Ms, execution_time
        
    return eig_vectors, Ys, Ms

In [19]:
## used to compare DBPCA with traditional PCA
def naive_pca_per_window_with_f(B, X, k, d, f, profiling=False):
    """
    B : int
        block size or window size
    X : numpy array n x d
        n is the number of data points and d is the number of attributes
    output q = estimated eigen vectors of cov(X)
    k : int
        number of principal components
    f : float
        forgetting factor (0,1], 1 basically means no forgetting factor
    """
    start_time = time.time()
    eig_vectors = list()
    Ys = list()
    Ms = list()
    n = X.shape[0]
    t0 = B
    for i in range(int(n/B)):
        start = i*B
        end = (i*B)+B
        delta_t = t0-i ## delta_t represent the age of the point
        W = X[start:end,:] * (math.pow(2, -f*delta_t))
        M = np.mean(W, axis=0) # first pass on X
        C = W - M              # second pass on X
        Y, eig_vector, k = do_pca(C) ## Y.T should be the same as reduced_X above
        Ys.append(Y)
        eig_vectors.append(eig_vector)
        Ms.append(M)
        
    if profiling:
        execution_time = time.time() - start_time ## return time in nanosecond
        return eig_vectors, Ys, Ms, execution_time
        
    return eig_vectors, Ys, Ms

In [20]:
M = np.mean(X, axis=0) # first pass on X
C = X - M              # second pass on X
Y, eig_vectors, k = do_pca(C) ## Y.T should be the same as reduced_X above
eig_vectors

array([[-0.42101852, -0.74305996, -0.52019737],
       [-0.69509016, -0.10415108,  0.71133833],
       [ 0.58274614, -0.66107069,  0.47264413]])

In [21]:
back_X = reconstruct_X(Y, eig_vectors, k).T + M
back_X

array([[1.68251986, 2.69206233, 0.71562236],
       [2.39334583, 2.21180288, 1.58854318],
       [2.94547341, 2.90643341, 1.45884194],
       ...,
       [1.04565381, 2.25293917, 0.66957513],
       [1.3170083 , 1.53965394, 4.31878756],
       [1.21058426, 2.55385917, 2.07470798]])

## How we can reconstruct one data point only, for example data point at index 9?

In [22]:
x_9 = X[9,:].reshape((-1,1))
y_9 = project_a_data_point(x_9, eig_vectors, k)

In [23]:
back_x_9 = reconstruct_x(y_9, eig_vectors, k) 
back_x_9

array([[2.09439562],
       [1.49333031],
       [2.62422045]])

In [24]:
x_9

array([[2.09439562],
       [1.49333031],
       [2.62422045]])

In [25]:
eig_vectors

array([[-0.42101852, -0.74305996, -0.52019737],
       [-0.69509016, -0.10415108,  0.71133833],
       [ 0.58274614, -0.66107069,  0.47264413]])